### Installing and importing the required modules

In [ ]:
!pip install torch transformers accelerate bitsandbytes datasets evaluate
!pip install -U bitsandbytes

In [ ]:
import torch
import numpy as np
import pandas as pd
from evaluate import load
from typing import Dict, Any
from datasets import Dataset
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BitsAndBytesConfig, TrainingArguments, Trainer

### Setting up the environment

In [ ]:
# Mounting the drive
drive.mount('/content/drive')

### Constants, hyperparameters and model configurations

In [ ]:
seed = 42 # Seed for reproducibility
test_size = 0.2 # Train-test split percentage
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # The device to run the model on
model_id = "distilbert-base-uncased" # The model ID of the Llama model
dataset_path = "/content/drive/MyDrive/Colab Notebooks/FineTuningLLM/datasets/iphone_products.csv" # The path to the dataset
model_path = "/content/drive/MyDrive/Colab Notebooks/FineTuningLLM/saved_models/iphone_products_classifier" # Path to save the trained model to

In [ ]:
# Print the detected device
print(f"Detected device: {device}")

### Data loading

In [ ]:
# Load the dataset into a pandas DataFrame
dataset = pd.read_csv(
    dataset_path,
    delimiter = ",",
    on_bad_lines = "skip"  # Skip problematic lines if necessary
)

In [ ]:
# Drop null values
dataset.dropna(inplace=True)

In [ ]:
# Show a subset of the samples
dataset.head()

### Tokenizer

In [ ]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, paddind_side="left")

### Preprocess data

In [ ]:
# Concatenate the product and its title
dataset["summary"] = dataset["product"] + " - " + dataset["title"]

In [ ]:
# Instantiate  the label encoder
label_encoder = LabelEncoder()

# Encode the target column (category_description) into numeric labels
dataset.loc[:, "label"] = label_encoder.fit_transform(dataset["label"])

# Extract and print the total number of classes
num_classes = len(label_encoder.classes_)
print(f"Total number of classes: {num_classes}")

In [ ]:
# Convert the Pandas DataFrame to a Hugging Face Dataset
hf_dataset = Dataset.from_pandas(dataset)

# Train-test split
train_dataset, test_dataset = hf_dataset.train_test_split(test_size=test_size, seed=seed).values()

In [ ]:
# Preprocess the dataset
def preprocess(examples: Dict[str, Any], max_length: int = 48) -> Dict[str, Any]:
    # Tokenize the input sequences
    return tokenizer(
        examples["summary"],
        truncation = True,
        padding = "max_length",
        max_length = max_length
    )

# Tokenize the dataset
tokenized_train_dataset = train_dataset.map(preprocess, batched=True)
tokenized_test_dataset = test_dataset.map(preprocess, batched=True)

# Remove unnecessary columns
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["product", "title"])
tokenized_test_dataset = tokenized_test_dataset.remove_columns(["product", "title"])

# Display the sequence length
print(f"Sequence length: {len(tokenized_train_dataset[0]['input_ids'])}")

### Building the model

In [ ]:
# Load the model
model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels = num_classes
)

In [ ]:
# Move the model to the taret device
model.to(device);

In [ ]:
# Display the model
model

### Trainig the model

In [ ]:
# Load the accuracy metric
accuracy_metric = load("accuracy")

# Define a custum function to compute the metrics
def compute_metrics(eval_pred: torch.Tensor) -> torch.Tensor:
    # Extract the logits and the lables from the output of the model
    logits, labels = eval_pred

    # Extract the predictions for each sample
    predictions = np.argmax(logits, axis=-1)

    # Compute and return the accuarcy
    return accuracy_metric.compute(predictions=predictions, references=labels)

In [ ]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir = "./iphone_products_classifier",
    eval_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = 3e-4,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    num_train_epochs = 20,
    weight_decay = 0.01,
    logging_dir = "./logs",
    logging_strategy = "steps",
    logging_steps = 10,
    save_total_limit = 2,
    load_best_model_at_end = True,
    metric_for_best_model = "accuracy",
    greater_is_better = True,
    report_to = "none",
    fp16 = True
)

In [ ]:
# Instantiate the trainer to train the model
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_train_dataset,
    eval_dataset = tokenized_test_dataset,
    compute_metrics = compute_metrics
)

# Trainin the model
trainer.train()

### Save the model

In [ ]:
# Saving the adapter to the destination path
model.save_pretrained(model_path)

### Load the fine-tuned model

In [ ]:
# Define the quantization configurations of the model (only for CUDA devices)
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype = torch.bfloat16,
    bnb_4bit_use_double_quant = True
)

In [ ]:
# Load the fine-tuned model
model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    low_cpu_mem_usage = True,
    quantization_config = quantization_config
)

In [ ]:
# Move the fine-tuned model to the target device
model.to(device);

### Inference

In [ ]:
# Tokenize a sample input
inputs = tokenizer(
    [
        "Iphone 12 pro 256 danneggiato", # 1
        "Cover antigraffio per iPhone", # 0
        "Drone per iphone con custodia", # 0
        "iPhone 13 mini 500TB Rosa", # 1
        "Set di pellicole per iphone 15 pro max" # 0
    ],
    padding = True,
    truncation = True,
    return_tensors="pt"
  ).to(device)

In [ ]:
# Perform inference
with torch.no_grad():
    # Compute the output of the model
    outputs = model(**inputs)

    # Extract the predictions
    predictions = torch.argmax(outputs.logits, dim=-1)

# Convert the predictions to a numpy array
predictions = predictions.cpu().numpy()

# Convert the predicted labels to the corresponding categories
predicted_categories = label_encoder.inverse_transform(predictions)

In [ ]:
# Display the predicted categories
for idx, predicted_category in enumerate(predicted_categories):
    print(f"Sample {idx + 1} --> Predicted label: {predictions[idx]} | Predicted Category: {predicted_category}")